In [28]:
!pip install pyngrok
!pip install flask_ngrok
!pip install HTMLResponse
!pip install dash pandas plotly
!pip install flask-ngrok
!pip install nest_asyncio

ERROR: Could not find a version that satisfies the requirement HTMLResponse (from versions: none)
ERROR: No matching distribution found for HTMLResponse


LECTURA DE DATA

In [24]:
import pandas as pd
data= pd.read_csv('/content/drive/Shareddrives/Ética y seguridad de Datos /Student_300MB_V2.csv')


In [48]:
users= pd.read_csv('/content/drive/Shareddrives/Ética y seguridad de Datos /Users.csv')


In [65]:
data.info()
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501836 entries, 0 to 2501835
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   student_id             int64  
 1   school                 object 
 2   sex                    object 
 3   age                    int64  
 4   address                object 
 5   famsize                object 
 6   Pstatus                object 
 7   Medu                   int64  
 8   Fedu                   int64  
 9   Mjob                   object 
 10  Fjob                   object 
 11  reason                 object 
 12  guardian               object 
 13  Dalc                   int64  
 14  Walc                   int64  
 15  health                 float64
 16  freetime               int64  
 17  famrel                 int64  
 18  romantic               object 
 19  nursery                object 
 20  higher                 object 
 21  internet               object 
 22  paid              

Ngrok authtoken extraído de : https://ngrok.com/

In [61]:
!ngrok authtoken  "2yOWunFQ5aFI8xxnnckVEPfDV90_WrRYEZFdASBCJi8kGBKG"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Cálculo de KPIS

In [55]:

# Cálculo de columnas útiles por curso
# ==============================

# Progreso por curso
data['progress_pct_mat'] = data.apply(lambda row: ((row['G3_mat'] - row['G1_mat']) / row['G1_mat']) if row['G1_mat'] != 0 else 0, axis=1)
data['progress_pct_por'] = data.apply(lambda row: ((row['G3_por'] - row['G1_por']) / row['G1_por']) if row['G1_por'] != 0 else 0, axis=1)

# Grupo de rendimiento
data['grupo_rendimiento_mat'] = data['G3_mat'].apply(lambda x: 'Aprobado' if x >= 10.5 else 'Desaprobado')
data['grupo_rendimiento_por'] = data['G3_por'].apply(lambda x: 'Aprobado' if x >= 10.5 else 'Desaprobado')

# Guardar df kpis por curso
df_kpis_mat = data.copy()
df_kpis_por = data.copy()

valid_registro_ids_mat = df_kpis_mat['registro_id'].unique()[:100]
valid_registro_ids_por = df_kpis_por['registro_id'].unique()[:100]

Data sensible a manejar

In [53]:
# === DATA SENSIBLE CIFRADA ===
campos_a_cifrar = [
    "school", "sex", "age", "address", "famsize", "Pstatus",
    "Medu", "Fedu", "Mjob", "Fjob", "guardian",
    "failures_mat", "absences", "G1_mat", "G2_mat", "G3_mat",
    "failures_por", "G1_por", "G2_por", "G3_por",
    "Dalc", "Walc", "goout", "romantic", "internet", "health"
]


Implementación

In [64]:
import nest_asyncio
from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import HTMLResponse
from pyngrok import ngrok
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import uvicorn

# === CONFIGURACIÓN ===

nest_asyncio.apply()

app = FastAPI()


# === RUTAS ===

# Exponer URL ngrok
public_url = ngrok.connect(8000)
print(f"FastAPI app is running at: {public_url}")
@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <html>
        <head>
            <title>Login - Sistema Educativo</title>
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        </head>
        <body>
            <div class="container mt-5">
                <h1>Bienvenido al Sistema Educativo</h1>
                <p>Por favor, ingresa tu Registro ID para continuar:</p>
                <form action="/login" method="get">
                    <div class="form-group">
                        <label for="registro_id">Registro ID:</label>
                        <input type="text" id="registro_id" name="registro_id" class="form-control" required>
                    </div>
                    <button type="submit" class="btn btn-primary mt-3">Ingresar</button>
                </form>
            </div>
        </body>
    </html>
    """

# Ruta login inicial
@app.get("/login", response_class=HTMLResponse)
async def login(registro_id: str = Query(...)):
    user_info = users[users['registro_id'] == registro_id]

    if user_info.empty:
        raise HTTPException(status_code=404, detail="Usuario no encontrado")

    role_prefix = registro_id[0].upper()

    if role_prefix == 'E':
        return f"""
        <html>
            <head><meta http-equiv="refresh" content="0; url=/student-dashboard?student_id={registro_id}&course=mathematics"></head>
            <body>Redirigiendo a Dashboard Estudiante...</body>
        </html>
        """
    elif role_prefix == 'P':
        return f"""
        <html>
            <head><meta http-equiv="refresh" content="0; url=/teacher-dashboard?registro_id={registro_id}&course=mathematics"></head>
            <body>Redirigiendo a Dashboard Profesor...</body>
        </html>
        """
    elif role_prefix == 'S':
        return f"""
        <html>
            <head><meta http-equiv="refresh" content="0; url=/supervisor-dashboard?registro_id={registro_id}&course=mathematics"></head>
            <body>Redirigiendo a Dashboard Supervisor...</body>
        </html>
        """
    elif role_prefix == 'A':
        return f"""
        <html>
            <head><meta http-equiv="refresh" content="0; url=/admin-dashboard?registro_id={registro_id}&course=mathematics"></head>
            <body>Redirigiendo a Dashboard Administrador...</body>
        </html>
        """
@app.get("/student-dashboard", response_class=HTMLResponse)
async def student_dashboard(student_id: str = Query(...), course: str = Query('mathematics')):

    # === Fase de prueba
    if course == 'mathematics':
        valid_registro_ids = valid_registro_ids_mat
    elif course == 'portuguese':
        valid_registro_ids = valid_registro_ids_por
    else:
        raise HTTPException(status_code=400, detail="Curso no válido")

    if student_id not in valid_registro_ids:
        raise HTTPException(status_code=400, detail="Estudiante no permitido en fase de prueba")

    # === Columnas según curso
    if course == 'mathematics':
        g1_col, g2_col, g3_col = 'G1_mat', 'G2_mat', 'G3_mat'
        studytime_col = 'studytime_mat'
        progress_col = 'progress_pct_mat'
    elif course == 'portuguese':
        g1_col, g2_col, g3_col = 'G1_por', 'G2_por', 'G3_por'
        studytime_col = 'studytime_por'
        progress_col = 'progress_pct_por'

    # === Variables del estudiante
    student_info = data[data['registro_id'] == student_id]
    if student_info.empty:
        raise HTTPException(status_code=404, detail="Estudiante no encontrado")

    g1 = student_info[g1_col].values[0]
    g2 = student_info[g2_col].values[0]
    g3 = student_info[g3_col].values[0]
    absences = student_info['absences'].values[0]
    progress_decimal = student_info[progress_col].values[0]
    progress_pct = round(progress_decimal * 100)

    # === Formulario para cambiar de curso
    form_html = f"""
    <form method="get" action="/student-dashboard">
        <input type="hidden" name="student_id" value="{student_id}">
        <div class="form-group">
            <label>Curso:</label>
            <select name="course" class="form-control">
                <option value="mathematics" {'selected' if course == 'mathematics' else ''}>Matemáticas</option>
                <option value="portuguese" {'selected' if course == 'portuguese' else ''}>Portugués</option>
            </select>
        </div>
        <button type="submit" class="btn btn-primary mt-3">Aplicar</button>
    </form>
    """

    # === Gráficos
    fig_g3 = px.histogram(data, x=g3_col, nbins=20,
                          title=f'¿Cómo se comparan tus resultados finales en {course.capitalize()}? 🎓',
                          template='plotly_white')
    fig_g3.add_vline(x=g3, line_dash='dash', line_color='red', annotation_text='Tu Nota', annotation_position='top')

    fig_progress = px.histogram(data, x=progress_col, nbins=30,
                                title='Tu avance en el curso (%)',
                                template='plotly_white')
    fig_progress.add_vline(x=progress_decimal, line_dash='dash', line_color='red', annotation_text=f'{progress_pct}%', annotation_position='top')

    fig_absences = px.histogram(data, x='absences', nbins=20,
                                title='¿Cómo se comparan tus ausencias?',
                                template='plotly_white')
    fig_absences.add_vline(x=absences, line_dash='dash', line_color='red', annotation_text='Tus Ausencias', annotation_position='top')

    # === Estadísticas para boxplot
    serie = data[g3_col]

    promedio = round(serie.mean(), 2)
    mediana = round(serie.median(), 2)
    maximo = round(serie.max(), 2)
    minimo = round(serie.min(), 2)
    q1 = round(serie.quantile(0.25), 2)
    q3 = round(serie.quantile(0.75), 2)

    # Boxplot
    fig_box = px.box(data, y=g3_col,
                     title='Distribución de Calificaciones',
                     template='plotly_white')
    fig_box.add_scatter(y=[g3], x=[0], mode='markers',
                        marker=dict(color='red', size=12),
                        name='Tu Nota')

    # Texto para ¡Te ayudo a entender tus notas!
    if g3 >= q3:
        analysis_text = "¡Excelente! Tu nota se encuentra entre las más altas del curso (cuartil superior). 🎓"
    elif g3 >= mediana:
        analysis_text = "¡Muy bien! Tu nota está por encima de la mediana del curso. Sigue así. ✅"
    elif g3 >= q1:
        analysis_text = "Tu nota está en el rango medio-bajo. Hay oportunidades de mejora. 💪"
    else:
        analysis_text = "Tu nota está entre las más bajas del curso. No te preocupes, con más estudio puedes mejorar. 🚀"

    # === Tabla resumen
    table_html = f"""
    <table class="table table-bordered">
        <thead class="thead-dark">
            <tr>
                <th>Componente</th>
                <th>Nota / Máximo</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>{g1_col}</td>
                <td>{g1} / 20</td>
            </tr>
            <tr>
                <td>{g2_col}</td>
                <td>{g2} / 20</td>
            </tr>
            <tr>
                <td>{g3_col}</td>
                <td>{g3} / 20</td>
            </tr>
            <tr>
                <td>Absences</td>
                <td>{absences}</td>
            </tr>
            <tr>
                <td>Progreso (%)</td>
                <td>{progress_pct}%</td>
            </tr>
        </tbody>
    </table>
    """

    # === HTML final
    return f"""
    <html>
        <head>
            <title>Dashboard Estudiante {student_id} - {course.capitalize()}</title>
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        </head>
        <body>
            <div class="container mt-5">
                <h1>Explorando tu recorrido académico 📚 ({course.capitalize()})</h1>
                <div class="card mt-4 p-3">
                    <h3>Opciones</h3>
                    {form_html}
                </div>
                <div class="card mt-4 p-3">
                    <h3>Resumen de tu desempeño</h3>
                    {table_html}
                </div>
                <div class="card mt-4">
                    <h3>¿Cómo se comparan tus resultados finales?</h3>
                    <div>{fig_g3.to_html(full_html=False)}</div>
                </div>
                <div class="card mt-4">
                    <h3>Tu avance en el curso (%)</h3>
                    <div>{fig_progress.to_html(full_html=False)}</div>
                </div>
                <div class="card mt-4">
                    <h3>¿Cómo se comparan tus ausencias?</h3>
                    <div>{fig_absences.to_html(full_html=False)}</div>
                </div>
                <div class="card mt-4">
                    <h3>Distribución de Calificaciones</h3>
                    <div>{fig_box.to_html(full_html=False)}</div>
                </div>
                <div class="card mt-4 p-3 bg-light">
                    <h3>¡Te ayudo a entender tus notas! 📝</h3>
                    <p style="font-size:16px;">{analysis_text}</p>
                </div>
            </div>
        </body>
    </html>
    """

@app.get("/teacher-dashboard", response_class=HTMLResponse)
async def teacher_dashboard(registro_id: str = Query(...), course: str = Query('mathematics'), student_id: str = Query(None)):

    # Elegir df_kpis y registro_id válidos
    if course == 'mathematics':
        df_kpis = df_kpis_mat
        valid_registro_ids = valid_registro_ids_mat
        g1_col, g2_col, g3_col = 'G1_mat', 'G2_mat', 'G3_mat'
        studytime_col = 'studytime_mat'
        progress_col = 'progress_pct_mat'
        group_col = 'grupo_rendimiento_mat'
    elif course == 'portuguese':
        df_kpis = df_kpis_por
        valid_registro_ids = valid_registro_ids_por
        g1_col, g2_col, g3_col = 'G1_por', 'G2_por', 'G3_por'
        studytime_col = 'studytime_por'
        progress_col = 'progress_pct_por'
        group_col = 'grupo_rendimiento_por'
    else:
        raise HTTPException(status_code=400, detail="Curso no válido")

    # Formulario HTML
    student_options = '\n'.join(
        [f'<option value="{sid}" {"selected" if sid == student_id else ""}>{sid}</option>' for sid in valid_registro_ids]
    )

    form_html = f"""
    <form method="get" action="/teacher-dashboard">
        <input type="hidden" name="registro_id" value="{registro_id}">
        <div class="form-group">
            <label>Curso:</label>
            <select name="course" class="form-control">
                <option value="mathematics" {'selected' if course == 'mathematics' else ''}>Matemáticas</option>
                <option value="portuguese" {'selected' if course == 'portuguese' else ''}>Portugués</option>
            </select>
        </div>
        <div class="form-group mt-2">
            <label>Filtrar por Student ID:</label>
            <select name="student_id" class="form-control">
                <option value="">-- TODOS --</option>
                {student_options}
            </select>
        </div>
        <button type="submit" class="btn btn-primary mt-3">Aplicar Filtros</button>
    </form>
    """

    # Si hay student_id → mostrar vista igual que student-dashboard
    if student_id:
        if student_id not in valid_registro_ids:
            raise HTTPException(status_code=400, detail="Estudiante no permitido en fase de prueba")

        student_info = df_kpis[df_kpis['registro_id'] == student_id]
        if student_info.empty:
            raise HTTPException(status_code=404, detail="Estudiante no encontrado")

        g1 = student_info[g1_col].values[0]
        g2 = student_info[g2_col].values[0]
        g3 = student_info[g3_col].values[0]
        absences = student_info['absences'].values[0]
        progress_decimal = student_info[progress_col].values[0]
        progress_pct = round(progress_decimal * 100)

        # Gráficos
        fig_g3 = px.histogram(df_kpis, x=g3_col, nbins=20,
                              title=f'¿Cómo se comparan sus resultados finales en {course.capitalize()}? 🎓',
                              template='plotly_white')
        fig_g3.add_vline(x=g3, line_dash='dash', line_color='red', annotation_text='Nota del estudiante', annotation_position='top')

        fig_progress = px.histogram(df_kpis, x=progress_col, nbins=30,
                                    title='Avance en el curso (%)',
                                    template='plotly_white')
        fig_progress.add_vline(x=progress_decimal, line_dash='dash', line_color='red', annotation_text=f'{progress_pct}%', annotation_position='top')

        fig_absences = px.histogram(df_kpis, x='absences', nbins=20,
                                    title='¿Cómo se comparan las ausencias?',
                                    template='plotly_white')
        fig_absences.add_vline(x=absences, line_dash='dash', line_color='red', annotation_text='Ausencias del estudiante', annotation_position='top')

        # Estadísticas para boxplot
        serie = df_kpis[g3_col]

        promedio = round(serie.mean(), 2)
        mediana = round(serie.median(), 2)
        maximo = round(serie.max(), 2)
        minimo = round(serie.min(), 2)
        q1 = round(serie.quantile(0.25), 2)
        q3 = round(serie.quantile(0.75), 2)

        # Boxplot
        fig_box = px.box(df_kpis, y=g3_col,
                         title='Distribución de Calificaciones',
                         template='plotly_white')

        # Agregar nota del estudiante como punto rojo
        fig_box.add_scatter(y=[g3], x=[0], mode='markers',
                            marker=dict(color='red', size=12),
                            name='Nota del estudiante')

        # Texto para sección ¡Te ayudo a entender tus notas!
        if g3 >= q3:
            analysis_text = "¡Excelente! Tu nota se encuentra entre las más altas del curso (cuartil superior). 🎓"
        elif g3 >= mediana:
            analysis_text = "¡Muy bien! Tu nota está por encima de la mediana del curso. Sigue así. ✅"
        elif g3 >= q1:
            analysis_text = "Tu nota está en el rango medio-bajo. Hay oportunidades de mejora. 💪"
        else:
            analysis_text = "Tu nota está entre las más bajas del curso. No te preocupes, con más estudio puedes mejorar. 🚀"

        # Tabla resumen
        table_html = f"""
        <table class="table table-bordered">
            <thead class="thead-dark">
                <tr>
                    <th>Componente</th>
                    <th>Nota / Máximo</th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <td>{g1_col}</td>
                    <td>{g1} / 20</td>
                </tr>
                <tr>
                    <td>{g2_col}</td>
                    <td>{g2} / 20</td>
                </tr>
                <tr>
                    <td>{g3_col}</td>
                    <td>{g3} / 20</td>
                </tr>
                <tr>
                    <td>Absences</td>
                    <td>{absences}</td>
                </tr>
                <tr>
                    <td>Progreso (%)</td>
                    <td>{progress_pct}%</td>
                </tr>
            </tbody>
        </table>
        """

        return f"""
        <html>
            <head>
                <title>Dashboard Profesor {registro_id} - {course.capitalize()}</title>
                <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
            </head>
            <body>
                <div class="container mt-5">
                    <h1>Dashboard Profesor - {course.capitalize()}</h1>
                    <div class="card mt-4 p-3">
                        <h3>Opciones de Filtro</h3>
                        {form_html}
                    </div>
                    <div class="card mt-4 p-3">
                        <h3>Resumen del desempeño del estudiante</h3>
                        {table_html}
                    </div>
                    <div class="card mt-4">
                        <h3>¿Cómo se comparan sus resultados finales?</h3>
                        <div>{fig_g3.to_html(full_html=False)}</div>
                    </div>
                    <div class="card mt-4">
                        <h3>Avance en el curso (%)</h3>
                        <div>{fig_progress.to_html(full_html=False)}</div>
                    </div>
                    <div class="card mt-4">
                        <h3>¿Cómo se comparan las ausencias?</h3>
                        <div>{fig_absences.to_html(full_html=False)}</div>
                    </div>
                    <div class="card mt-4">
                        <h3>Distribución de Calificaciones</h3>
                        <div>{fig_box.to_html(full_html=False)}</div>
                    </div>
                    <div class="card mt-4 p-3 bg-light">
                        <h3>¡Te ayudo a entender tus notas! 📝</h3>
                        <p style="font-size:16px;">{analysis_text}</p>
                    </div>
                </div>
            </body>
        </html>
        """

    # Si no hay student_id → KPIs globales
    else:
        df_filtered = df_kpis[df_kpis['registro_id'].isin(valid_registro_ids)]

        fig_abs_vs_g3 = px.scatter(df_filtered, x='absences', y=g3_col, trendline='ols',
                                   title='Tasa de Absentismo vs Rendimiento Académico', template='plotly_white')

        fig_study_vs_g3 = px.box(df_filtered, x=studytime_col, y=g3_col,
                                 title='Relación entre Tiempos de Estudio y Notas (G3)', template='plotly_white')

        fig_group_perf = px.box(df_filtered, x=group_col, y=g3_col,
                                title='Rendimiento Promedio por Grupo de Estudiantes', template='plotly_white')

        return f"""
        <html>
            <head>
                <title>Dashboard Profesor {registro_id} - {course.capitalize()}</title>
                <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
            </head>
            <body>
                <div class="container mt-5">
                    <h1>Dashboard Profesor - {course.capitalize()}</h1>
                    <div class="card mt-4 p-3">
                        <h3>Opciones de Filtro</h3>
                        {form_html}
                    </div>
                    <div class="card mt-4">
                        <h3>Tasa de Absentismo vs Rendimiento Académico</h3>
                        <div>{fig_abs_vs_g3.to_html(full_html=False)}</div>
                    </div>
                    <div class="card mt-4">
                        <h3>Relación entre Tiempos de Estudio y Notas (G3)</h3>
                        <div>{fig_study_vs_g3.to_html(full_html=False)}</div>
                    </div>
                    <div class="card mt-4">
                        <h3>Rendimiento Promedio por Grupo de Estudiantes</h3>
                        <div>{fig_group_perf.to_html(full_html=False)}</div>
                    </div>
                </div>
            </body>
        </html>
        """

# === Ruta supervisor-dashboard ===
@app.get("/supervisor-dashboard", response_class=HTMLResponse)
async def supervisor_dashboard(registro_id: str = Query(...), course: str = Query('mathematics')):

    if course == 'mathematics':
        df_kpis = df_kpis_mat
        valid_registro_ids = valid_registro_ids_mat
    elif course == 'portuguese':
        df_kpis = df_kpis_por
        valid_registro_ids = valid_registro_ids_por
    else:
        raise HTTPException(status_code=400, detail="Curso no válido")

    # Filtrar solo los primeros 10 registro_id válidos
    df_filtered = df_kpis[df_kpis['registro_id'].isin(valid_registro_ids)].copy()

    # Eliminar columnas cifradas
    df_supervisor = df_filtered.drop(columns=campos_a_cifrar, errors='ignore')

    # Convertir a tabla HTML
    table_html = df_supervisor.to_html(classes='table table-striped', index=False)

    # Formulario cambio de curso
    form_html = f"""
    <form method="get" action="/supervisor-dashboard">
        <input type="hidden" name="registro_id" value="{registro_id}">
        <div class="form-group">
            <label>Curso:</label>
            <select name="course" class="form-control">
                <option value="mathematics" {'selected' if course == 'mathematics' else ''}>Matemáticas</option>
                <option value="portuguese" {'selected' if course == 'portuguese' else ''}>Portugués</option>
            </select>
        </div>
        <button type="submit" class="btn btn-primary mt-3">Aplicar</button>
    </form>
    """

    return f"""
    <html>
        <head>
            <title>Dashboard Supervisor {registro_id} - {course.capitalize()}</title>
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        </head>
        <body>
            <div class="container mt-5">
                <h1>Dashboard Supervisor - {course.capitalize()}</h1>
                <div class="card mt-4 p-3">
                    <h3>Opciones</h3>
                    {form_html}
                </div>
                <div class="card mt-4 p-3">
                    <h3>Data Cruda (sin información sensible)</h3>
                    {table_html}
                </div>
            </div>
        </body>
    </html>
    """

# === Ruta admin-dashboard ===
@app.get("/admin-dashboard", response_class=HTMLResponse)
async def admin_dashboard(registro_id: str = Query(...), course: str = Query('mathematics')):

    if course == 'mathematics':
        df_kpis = df_kpis_mat
        valid_registro_ids = valid_registro_ids_mat
    elif course == 'portuguese':
        df_kpis = df_kpis_por
        valid_registro_ids = valid_registro_ids_por
    else:
        raise HTTPException(status_code=400, detail="Curso no válido")

    # Filtrar solo los primeros 10 registro_id válidos
    df_filtered = df_kpis[df_kpis['registro_id'].isin(valid_registro_ids)].copy()

    # Admin puede ver TODO
    table_html = df_filtered.to_html(classes='table table-striped', index=False)

    # Formulario cambio de curso
    form_html = f"""
    <form method="get" action="/admin-dashboard">
        <input type="hidden" name="registro_id" value="{registro_id}">
        <div class="form-group">
            <label>Curso:</label>
            <select name="course" class="form-control">
                <option value="mathematics" {'selected' if course == 'mathematics' else ''}>Matemáticas</option>
                <option value="portuguese" {'selected' if course == 'portuguese' else ''}>Portugués</option>
            </select>
        </div>
        <button type="submit" class="btn btn-primary mt-3">Aplicar</button>
    </form>
    """

    return f"""
    <html>
        <head>
            <title>Dashboard Administrador {registro_id} - {course.capitalize()}</title>
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        </head>
        <body>
            <div class="container mt-5">
                <h1>Dashboard Administrador - {course.capitalize()}</h1>
                <div class="card mt-4 p-3">
                    <h3>Opciones</h3>
                    {form_html}
                </div>
                <div class="card mt-4 p-3">
                    <h3>Data Completa</h3>
                    {table_html}
                </div>
            </div>
        </body>
    </html>
    """

# Run
if __name__ == '__main__':
    uvicorn.run(app, host="0.0.0.0", port=8000)




FastAPI app is running at: NgrokTunnel: "https://6055-34-138-228-242.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [315]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     132.251.2.246:0 - "GET / HTTP/1.1" 200 OK
INFO:     132.251.2.246:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     132.251.2.246:0 - "GET /login?registro_id=E00000003 HTTP/1.1" 200 OK
INFO:     132.251.2.246:0 - "GET /student-dashboard?student_id=E00000003&course=mathematics HTTP/1.1" 200 OK
INFO:     132.251.2.246:0 - "GET /student-dashboard?student_id=E00000003&course=portuguese HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [315]
